In [1]:
import pandas as pd
import numpy as np
file_to_load="purchase_data.csv"
purchase_data=pd.read_csv(file_to_load)
purchase_df=pd.DataFrame(purchase_data)

In [2]:
purchase_df.head(5)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
totalplayers = len(pd.unique(purchase_df['SN']))
print ("Total number of unique players is: ", totalplayers)

Total number of unique players is:  576


In [4]:
unique_items = purchase_df['Item ID'].nunique()
mean_price = purchase_df["Price"].mean()
purchase_vol = purchase_df["Purchase ID"].nunique()
total_revenue = purchase_df["Price"].sum()

finanalysis = pd.DataFrame({"Unique Items Purchased": [unique_items], "Mean Purchase Price": [mean_price], 
                            "Total Purchase Volume": [purchase_vol], "Total Purchase Revenue": [total_revenue]})

pd.options.display.float_format = '${:,.2f}'.format

finanalysis #Needs formatting for final dataframe; drop index column?

,Unique Items Purchased,Mean Purchase Price,Total Purchase Volume,Total Purchase Revenue
0,179,$3.05,780,"$2,379.77"


In [5]:
users_df = purchase_df.loc[:, ['SN', 'Age', 'Gender']]
users_df = users_df.drop_duplicates()

gendernums = users_df['Gender'].value_counts()
genderpercent = gendernums / (len(users_df['SN'])) * 100
gender_df = pd.DataFrame({'Gender Total':gendernums, 'Percent of All Players':genderpercent})

pd.options.display.float_format = '{:,.2f}%'.format

index=gender_df.index
index.name = "Gender"

gender_df

,Gender Total,Percent of All Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [6]:

gender_moneycount = purchase_df.groupby(["Gender"]).count()["Price"]
gender_moneyvol = purchase_df.groupby(["Gender"]).sum()["Price"]
gender_avgprice = purchase_df.groupby(["Gender"]).mean()["Price"]
gender_pricebyuser = gender_moneyvol / gender_df["Gender Total"]

money_gender = pd.DataFrame({"Number of Purchases":gender_moneycount, "Total Purchase Volume":gender_moneyvol, 
                          "Average Purchase Price":gender_avgprice, "Price Per User":gender_pricebyuser})

money_gender["Total Purchase Volume"] =  money_gender["Total Purchase Volume"].map("${:,.2f}".format)
money_gender["Average Purchase Price"] =  money_gender["Average Purchase Price"].map("${:,.2f}".format)
money_gender["Price Per User"] =  money_gender["Price Per User"].map("${:,.2f}".format)

money_gender = money_gender.loc[:,["Number of Purchases", "Total Purchase Volume", "Average Purchase Price", "Price Per User"]]

index=money_gender.index
index.name = "Gender"

money_gender

,Number of Purchases,Total Purchase Volume,Average Purchase Price,Price Per User
Gender,,,,
Female,113,$361.94,$3.20,$4.47
Male,652,"$1,967.64",$3.02,$4.07
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


In [7]:
# Set age bins up to max age (45)
bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
binlabel = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
users_df["Age Bracket"] = pd.cut(users_df["Age"], bins, labels = binlabel)

agetotals = users_df["Age Bracket"].value_counts()
agepercents = agetotals / (len(users_df['SN'])) * 100

age_df = pd.DataFrame({"Players Per Bracket": agetotals, "Percent of All Players": agepercents})
age_df["Percent of All Players"] = age_df["Percent of All Players"].map("{:,.2f}%".format)

#Sort the data frame by index, in this case the bins that have been defined as Age Brackets
index=age_df.index
index.name = "Age"

age_df.sort_index()


,Players Per Bracket,Percent of All Players
Age,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [8]:
# Purchase data into age bins
purchase_df["Age Bracket"] = pd.cut(purchase_df["Age"], bins, labels = binlabel)

bin_moneycount = purchase_df.groupby(["Age Bracket"]).count()["Price"]
bin_moneyvol = purchase_df.groupby(["Age Bracket"]).sum()["Price"]
bin_avgprice = purchase_df.groupby(["Age Bracket"]).mean()["Price"]
bin_byuser = bin_moneyvol / age_df["Players Per Bracket"]

money_age = pd.DataFrame({"Number of Purchases":bin_moneycount, "Total Purchase Volume":bin_moneyvol, 
                          "Average Purchase Price":bin_avgprice, "Price Per User":bin_byuser})

money_age["Total Purchase Volume"] =  money_age["Total Purchase Volume"].map("${:,.2f}".format)
money_age["Average Purchase Price"] =  money_age["Average Purchase Price"].map("${:,.2f}".format)
money_age["Price Per User"] =  money_age["Price Per User"].map("${:,.2f}".format)

money_age = money_age.loc[:,["Number of Purchases", "Total Purchase Volume", "Average Purchase Price", "Price Per User"]]

index=money_age.index
index.name = "Age Bracket"

money_age.sort_index()

,Number of Purchases,Total Purchase Volume,Average Purchase Price,Price Per User
Age Bracket,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,"$1,114.06",$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
40+,13,$38.24,$2.94,$3.19


In [9]:
user_purcount = purchase_df.groupby(["SN"]).count()["Price"]
user_avgpur = purchase_df.groupby(["SN"]).mean()["Price"]
user_totalpur = purchase_df.groupby(["SN"]).sum()["Price"]

spenders = pd.DataFrame({"# of Purchases":user_purcount, "Avg. Purchase Price":user_avgpur, "Total Spent": user_totalpur})

spenders = spenders.sort_values("Total Spent", ascending = False)

spenders["Avg. Purchase Price"] =  spenders["Avg. Purchase Price"].map("${:,.2f}".format)
spenders["Total Spent"] =  spenders["Total Spent"].map("${:,.2f}".format)

spenders.head(5)

,# of Purchases,Avg. Purchase Price,Total Spent
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.41,$13.62
Iskadarya95,3,$4.37,$13.10


In [12]:
sale_count = purchase_df.groupby(["Item ID", "Item Name"]).count()["Price"]
sale_vol = purchase_df.groupby(["Item ID", "Item Name"]).sum()["Price"]
sale_avg = purchase_df.groupby(["Item ID", "Item Name"]).mean()["Price"]
#Attempting to find the best way to pull the price of the item, I realized that the prices are not stable as this line
#returns a list. Therefore, using .mean() would be better.
#sale_price = purchase_df.groupby(["Item ID", "Item Name"])["Price"].unique()

bestsellers = pd.DataFrame({"Total Sold": sale_count, "Avg. Sale Price": sale_avg, "Revenue Earned": sale_vol})

bestsellers = bestsellers.sort_values("Total Sold", ascending = False)

bestsellers["Avg. Sale Price"] = bestsellers["Avg. Sale Price"].map("${:,.2f}".format)
bestsellers["Revenue Earned"] =  bestsellers["Revenue Earned"].map("${:,.2f}".format)

bestsellers.head(5)

,,Total Sold,Avg. Sale Price,Revenue Earned
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [14]:
moneymakers =  pd.DataFrame({"Revenue Earned": sale_vol, "Total Sold": sale_count, "Avg. Sale Price": sale_avg})

moneymakers = moneymakers.sort_values("Revenue Earned", ascending = False)

moneymakers["Avg. Sale Price"] = moneymakers["Avg. Sale Price"].map("${:,.2f}".format)
moneymakers["Revenue Earned"] =  moneymakers["Revenue Earned"].map("${:,.2f}".format)

moneymakers.head(5)

,,Revenue Earned,Total Sold,Avg. Sale Price
Item ID,Item Name,,,
92,Final Critic,$59.99,13,$4.61
178,"Oathbreaker, Last Hope of the Breaking Storm",$50.76,12,$4.23
82,Nirvana,$44.10,9,$4.90
145,Fiery Glass Crusader,$41.22,9,$4.58
103,Singed Scalpel,$34.80,8,$4.35
